In [ ]:
import pyeq3

In [ ]:
data = '''
  X        Y
5.357    10.376
5.457    10.489
5.797    10.874
5.936    11.049
6.161    11.327
6.697    12.054
6.731    12.077
6.775    12.138
8.442    14.744
9.769    17.068
9.861    17.104
'''

In [ ]:
# see IModel.fittingTargetDictionary
equation = pyeq3.Models_2D.Polynomial.Linear('SSQABS') # straight line
#equation = pyeq3.Models_2D.Exponential.Exponential('SSQABS', 'Offset') # exponential with offset

In [ ]:
 # check for functions requiring non-zero nor non-negative data such as 1/x, etc.
try:
    pyeq3.dataConvertorService().ConvertAndSortColumnarASCII(data, equation, False)
except:
    raise Exception(equation.reasonWhyDataRejected)

In [ ]:
equation.Solve()
equation.CalculateModelErrors(equation.solvedCoefficients, equation.dataCache.allDataCacheDictionary)
equation.CalculateCoefficientAndFitStatistics()

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy, scipy

# raw data for scatterplot and data limits
x_data = equation.dataCache.allDataCacheDictionary['IndependentData'][0]
y_data = equation.dataCache.allDataCacheDictionary['DependentData']

# now create data for the fitted equation plot
xModel = numpy.linspace(min(x_data), max(x_data))

tempcache = equation.dataCache
equation.dataCache = pyeq3.dataCache()
equation.dataCache.allDataCacheDictionary['IndependentData'] = numpy.array([xModel, xModel])
equation.dataCache.FindOrCreateAllDataCache(equation)
yModel = equation.CalculateModelPredictions(equation.solvedCoefficients, equation.dataCache.allDataCacheDictionary)
equation.dataCache = tempcache

fig = plt.figure()
ax = fig.add_subplot(1,1,1)

# first the raw data as a scatter plot
ax.plot(x_data, y_data,  'D')

# now the model as a line plot
ax.plot(xModel, yModel)

# now calculate confidence intervals
# http://support.sas.com/documentation/cdl/en/statug/63347/HTML/default/viewer.htm#statug_nlin_sect026.htm
# http://www.staff.ncl.ac.uk/tom.holderness/software/pythonlinearfit
mean_x = numpy.mean(x_data)			# mean of x
n = equation.nobs		    # number of samples in the origional fit

t_value = scipy.stats.t.ppf(0.975, equation.df_e) # (1.0 - (a/2)) is used for two-sided t-test critical value, here a = 0.05

confs = t_value * numpy.sqrt((equation.sumOfSquaredErrors/equation.df_e)*(1.0/n + (numpy.power((xModel-mean_x),2.0)/
                            ((numpy.sum(numpy.power(x_data,2)))-n*(numpy.power(mean_x,2.0))))))

# get lower and upper confidence limits based on predicted y and confidence intervals
upper = yModel + abs(confs)
lower = yModel - abs(confs)

# mask off any numbers outside the existing plot limits
booleanMask = yModel > matplotlib.pyplot.ylim()[0]
booleanMask &= (yModel < matplotlib.pyplot.ylim()[1])

# color scheme improves visibility on black background lines or points
ax.plot(xModel[booleanMask], lower[booleanMask], linestyle='solid', color='white')
ax.plot(xModel[booleanMask], upper[booleanMask], linestyle='solid', color='white')
ax.plot(xModel[booleanMask], lower[booleanMask], linestyle='dashed', color='blue')
ax.plot(xModel[booleanMask], upper[booleanMask], linestyle='dashed', color='blue')

ax.set_title('Model with 95 percent confidence intervals') # add a title
ax.set_xlabel('X Data') # X axis data label
ax.set_ylabel('Y Data') # Y axis data label

plt.tight_layout() # prevents cropping axis labels